In [1]:
#example読み込み
import pandas as pd
from prepro.data_builder import _format_to_bert3
path = '/Users/shota/Documents/ginza/wikihow_japanese/data/output/test.jsonl'
test_data = pd.read_json(path, orient='records', lines=True)
src = test_data['src'][0]
print(src)
tgt = test_data['tgt'][0]
print(tgt)

In [4]:
#文ごとに分かち書き
src = src.split('。') 
src = [a+ '。' for a in src]
tgt = tgt.split('。') 
tgt = [a+ '。' for a in tgt]
print(tgt[:-1])

['ガラガラヘビから離れましょう。', '医療処置を受けましょう。', '決して患部を心臓よりも高い位置に置いてはいけません。', '体を動かさずにじっとしましょう。']


In [19]:
# パラメータ
class bert_args():
    def __init__(self,min_src_ntokens = 5, max_src_ntokens = 200, min_nsents = 3, max_nsents = 1000, bert_config_path = None,temp_dir = None, encoder = 'classifier',param_init = 0,param_init_glorot = True):
        self.min_src_ntokens = min_src_ntokens
        self.max_src_ntokens = max_src_ntokens
        self.min_nsents = min_nsents
        self.max_nsents = max_nsents
        self.temp_dir = '../temp'
        self.bert_config_path = '../../Japanese_L-12_H-768_A-12_E-30_BPE_transformers/config.json'
        self.encoder = encoder
        self.param_init = param_init 
        self.param_init_glorot = param_init_glorot
        
args = bert_args()

In [20]:
args.min_src_ntokens

5

In [27]:
#モデルの定義
import torch
from pytorch_pretrained_bert import BertConfig
from models.model_builder import Summarizer
test_from  = '../models/bert_classifier/model_step_10000.pt'
checkpoint = torch.load(test_from, map_location=torch.device('cpu'))
config = BertConfig.from_json_file(args.bert_config_path)
model = Summarizer(args, device="cpu", load_pretrained_bert=False, bert_config = config)
model.load_cp(checkpoint)
model.eval()

Summarizer(
  (bert): Bert(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(32006, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): BertLayerNorm()
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (LayerNorm): BertLayerNorm()
                (d

In [75]:
#予測
rom models.data_loader import Batch,DataIterator
pre_src = b_data_dict['src']
pre_segs = b_data_dict['segs']
pre_labels = b_data_dict['labels']
pre_clss = b_data_dict['clss']

src = torch.tensor([pre_src])

labels = torch.tensor([pre_labels])
segs = torch.tensor([pre_segs])
mask = ~(src == 0)

clss = torch.tensor([pre_clss])
mask_cls = ~(clss == -1)
clss[clss == -1] = 0
sent_scores, mask = model(src, segs, clss, mask, mask_cls)

In [76]:
sent_scores

tensor([[1.4911e-03, 7.5662e-03, 2.2614e-03, 1.3349e-02, 3.2538e-04, 8.6739e-05,
         1.0555e-03, 5.7114e-03, 2.8742e-02, 1.9345e-03, 3.4621e-01, 5.1229e-04,
         4.8262e-04, 3.5451e-03, 1.6274e-03]], grad_fn=<SqueezeBackward1>)

In [77]:
#予測結果ソート
import numpy as np
sent_scores = sent_scores + mask.float()
sent_scores = sent_scores.cpu().data.numpy()
selected_ids = np.argsort(-sent_scores, 1)

In [84]:
#平滑化
def flatten_2d(data):
    for block in data:
        for elem in block:
            yield elem
selected_ids = list(flatten_2d(selected_ids))
print(selected_ids)

[10, 8, 3, 1, 7, 13, 2, 9, 14, 0, 6, 11, 12, 4, 5]


In [95]:
#上位3文抽出
src_str = b_data_dict['src_txt']
_pred = []
for i, idx in enumerate(selected_ids):

    if(len(src_str[i])==0):
        continue
    
    candidate = src_str[idx]    
    _pred.append(candidate)

    if len(_pred) == 3:
        break

In [105]:
#予測結果出力
print('予測結果')
print(_pred)
tgt_str = b_data_dict['tgt_txt']
print('答え')
print(tgt[:-1])

予測結果
['噛 ま れ た 箇 所 を 心 臓 よ り も 上 に 置 く と 、 毒 を 含 ん だ 血 液 が 猛 ス ピ ー ド で 心 臓 に 流 れ 込 み ま す 。', 'み な さ ん に 噛 み つ い た 蛇 が ガ ラ ガ ラ ヘ ビ か ど う か が 分 か ら な い 場 合 で も 、 す ぐ に 病 院 へ 直 行 し ま し ょ う 。', 'で き る 限 り 速 や か に 医 療 処 置 を 求 め る こ と が 大 切 で す 。']
答え
ガラガラヘビから離れましょう。医療処置を受けましょう。決して患部を心臓よりも高い位置に置いてはいけません。体を動かさずにじっとしましょう
